In [3]:
import pandas, numpy, os, re

In [4]:
article_directory = './hw01-data-raw/News Articles'
article_list = os.listdir(article_directory) # create list of files
article_list[:5]

['1101243072871.txt',
 '1101163341619.txt',
 '1101163941020.txt',
 '1101163268260.txt',
 '1101163374885.txt']

In [9]:
keywords = { 
    'mad cow': ['mad cow', 'prion', 'creutzfeldt-jakob', 'spongiform', 'encephalopath'], 
    'laboratory': ['lab'],
    'whistleblower': ['whistle'],
    'key names': ['Ryker', 'Sanderson', 'Boynton', 'Dortmund', 'Sulfate'],
    'Laurie/Laurel Sulfate': ['Sulfate', 'Laurie', 'Laurel'],
    'animal feed': ['feed'],
    'cash flow': ['$', 'dollar', 'money', 'cash', 'expense']
}
keywords.keys()

dict_keys(['mad cow', 'laboratory', 'whistleblower', 'key names', 'Laurie/Laurel Sulfate', 'animal feed', 'cash flow'])

regular expression from https://stackoverflow.com/questions/17268958/finding-occurrences-of-a-word-in-a-string-in-python-3

In [10]:
article_data = []
# create dataframe from list of dictionaries
# each dictionary: {filename: filename.txt, title: "", author: {author name}, date: {date posted}, ...}

for file_name in article_list:
    # open the file
    article = open(f'{article_directory}/{file_name}', encoding='ISO 8859-1')
    article_text = article.read()
    # replace weird characters and newlines with single space, convert to lower case
    # article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', ' ')
    # article_text = re.sub(' +', ' ', article_text).lower()

    # empty list of article keywords to populate
    article_keywords = []
    article_dict = { 'filename': file_name}
    # get other metadata for article_dict from Emi's code (article title, author, date published)
    article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', '#')
    article_text = re.sub(' +', ' ', article_text).lower()
    article_text = re.sub('(#)+', '#', article_text)
    article_text = article_text[1:]
    article_pieces =  article_text.split('#')
    # print(article_pieces)
    title = article_pieces[0]
    author = False
    date = False
    if "story by:" in article_pieces[1]:
        author = article_pieces[1][10:]
        date = article_pieces[2][23:]
    elif article_pieces[0] == " ":
        date = article_pieces[1].strip()[23:]
    else:
        date = article_pieces[2].strip()[23:]
    article_dict['title'] = title
    article_dict['date'] = date
    article_dict['author'] = author
    # tally up occurrences of each keyword category
    tally = 0
    for keyword in keywords.keys():
        article_dict[keyword] = 0
        contains_keyword = False
        for synonym in keywords[keyword]:
            num_found = sum(1 for _ in re.finditer(r'%s' % re.escape(synonym.lower()), article_text))
            article_dict[keyword] += num_found
            tally+=num_found
    if tally>0:
        article_data.append(article_dict)

In [14]:
article_df = pandas.DataFrame(article_data)
article_df['date'] = pandas.to_datetime(article_df['date'], format='mixed')
# article_df.sort_values(by='key names', ascending=False)
param = 'Laurie/Laurel Sulfate'
article_df[article_df[param] > 0].sort_values(by='date')

,filename,title,date,author,mad cow,laboratory,whistleblower,key names,Laurie/Laurel Sulfate,animal feed,cash flow
521,1101243338500.txt,uthor announces biotech lab to locate in alder...,2002-02-02,mel sacher,4,8,0,6,2,0,4
302,1101240332919.txt,boynton laboratories press release,2002-09-15,john panni,9,4,0,6,2,0,0
256,1101163018612.txt,boynton labs chosen for national testing,2003-09-13,ellie olmsen,7,7,0,5,2,0,0
125,1101243423748.txt,obituaries,2004-01-20,False,0,1,0,0,1,0,0
281,1101163840166.txt,synthetic prion causes neurological disease in...,2004-01-20,mel sacher,12,1,0,7,2,0,0
112,1101163595600.txt,alderwood lab breakthrough in mad cow disease,2004-03-02,john panni,15,4,0,10,3,0,0
163,1101163356500.txt,fda starts investigation of boynton,2004-04-16,john panni,4,5,0,5,2,0,0
91,1101163356001.txt,torch scandal?,2004-04-30,john panni,0,3,0,3,3,0,0
210,1101163308494.txt,ahs names top grads,2004-05-13,False,0,0,0,0,1,0,0
230,1101163977242.txt,fda cancels investigation,2004-06-19,mel sacher,0,3,0,7,3,0,0
